# Geovisualization Lab

## Group: &lt; Lab Group Number here &gt;

### Members: &lt; Members of the group here &gt;


In this this installment of our group labs we will be analyzing and visualizing geographic data; i.e., data that refers to geospatial entities. Geospatial entities can, for example, be particular places such as schools and libraries or political boundaries of cities or countries. Of course, this tutorial only scratches the surface. Consider this as a teaser into geovisualization, which in itself has become a branch of research and practice at the intersection of geography and visualization. We will only touch on a few basic steps to get your feet wet and hands dirty.


In [1]:
import altair as alt
import pandas as pd

### Install packages

For this tutorial we will continue to rely on Altair and Pandas, but add **GeoPandas**, which will help us to work with DataFrames that contain spatial entities to carry out geometric analysis on them. As before, the pip install command is carried out via the shell, which is indicated by the exclamation mark at the beginning of the line:

In [2]:
!pip install geopandas
import geopandas as gpd

To access the data of the OpenStreetMap, we will install the handy package **OSMPythonTools**:

In [3]:
!pip install OSMPythonTools

Once we have the tools assembled, we can get started working with geospatial data. There are actually plenty of formats used to record geospatial data, but GeoJSON has become an important standard for exchanging geospatial data on the web. However, please note that GeoPandas can actually load many other vector-based data formats used in digital cartography, such as shapefiles and GeoPackage.

### Import GeoJSON

Suppose we would like to get the geographic boundaries of Austin's neighborhoods. Akin to how we would read a JSON file with Pandas, we can also use `read_file()` provided by GeoPandas simply by passing the geojson filename and get a geographic DataFrame back:

In [4]:
neighborhoods = gpd.read_file("https://gist.githubusercontent.com/TieJean/40e9ccc69f0cc65b8e06ccf3fd60a3f0/raw/04ec5f998e63185e65f4d415827f1838ff8a25ab/austin.geojson")

The main difference between a regular Pandas DataFrame is that a GeoDataFrame features a `geometry` column, which is a geoseries containing the points, paths, and polygons for each row. For example, if each row represents one neighborhood, the respective geometries would probably contain the geospatial boundaries…

💡 *Are you curious what the neighborhoods dataframe actually looks like? Take a look at it with the methods you know by now:* 

In [5]:
neighborhoods.head()

,name,cartodb_id,created_at,updated_at,geometry
0,Blackland,1,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.72409 30.27926, -97.72514 ..."
1,Bouldin Creek,2,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.75962 30.24211, -97.76031 ..."
2,Brentwood,3,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.72354 30.33038, -97.72371 ..."
3,Cherrywood,4,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.70711 30.28920, -97.70700 ..."
4,Chestnut,5,2013-02-17T09:28:09.692000+00:00,2013-02-17T09:28:09.956000+00:00,"MULTIPOLYGON (((-97.71991 30.27379, -97.72010 ..."


Geographically speaking, the districts are defined by their geographic shapes, which are represented as polygons, each of which is a list of tuples of geographic coordinates. Next we add information about schools in Austin:

In [6]:
schools = gpd.read_file("https://gist.githubusercontent.com/TieJean/4e9f595eb3cb5dc60a960c3a499f6141/raw/171118ec54c599ce4bf9a30f9ab176427a47629f/austin_schools.geojson")
schools.head()

,FID,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,...,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR,geometry
0,81891,480000811280,4800008,ROOSTER SPRINGS EL,48,1001 BELTERRA DR,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-97.98292 30.19218)
1,81892,480000813086,4800008,SYCAMORE SPRINGS EL,48,14451 SAWYER RANCH RD,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-98.00025 30.17858)
2,81893,480000813151,4800008,SYCAMORE SPRINGS MIDDLE,48,14451 SAWYER RANCH RD,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-98.00025 30.17858)
3,81942,480001609410,4800016,AUSTIN CAN ACADEMY,48,2406 ROSEWOOD AVE,AUSTIN,TX,78702,48,...,1,N,N,N,N,4825,48046,48014,2018-2019,POINT (-97.70895 30.27222)
4,82016,480004408055,4800044,WAYSIDE EDEN PARK ACADEMY,48,6215 MANCHACA RD,AUSTIN,TX,78745,48,...,1,N,N,N,N,4821,48049,48014,2018-2019,POINT (-97.80292 30.20907)


💡 *Have a look at the schools as well, and compare the contents of the `geometry` columns in schools and districts. Do you notice anything?*

### Query OpenStreetMap

OpenStreetMap (OSM) is "a collaborative project to create a free editable map of the world". As such it has millions of contributing users who have been collecting, updating and refining map data for over 15 years, which has generated a vastly comprehensive source of geographic data. It is by no means complete—whatever this would mean—but it is an impressively large geographic database and, of course, a map in itself, too.

To get a list of libraries in Austin (according to the OSM), we first need to find the right Austin. For this we use the geocoding powers of OSM through the `Nominatim` search service:

In [7]:
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
city = nominatim.query('Austin, Texas')
city.areaId()

3600113314

OpenStreetMap has its own kind of query language, which is quite compact and can also be a source for errors. To make query formulation easier, you can either use the web interface [overpass turbo](http://overpass-turbo.eu) or the `overpassQueryBuilder`, which provides access to the main parameters:

In [8]:
from OSMPythonTools.overpass import overpassQueryBuilder

library_query = overpassQueryBuilder(
    area=3600113314, # the original field is city.areaId() but there is a weird bug in google colab which make Austin's cityID to be None. So I replace city.areaID() directly with its value. the query can be contrained by an area of an item
    elementType='node', # which are points (OSM also has ways and relations)
    # the selector in the next line is really the heart of the query:
    selector='"amenity"="library"', # we're looking for libraries
    out='body', # body indicates that we want the data, not just the count
    includeGeometry=True # and we want the geometric information, too
)

library_query

'area(3600113314)->.searchArea;(node["amenity"="library"](area.searchArea);); out body geom;'

The output of above cell is the compact version of the query, which is carried out in the next step:



In [9]:
from OSMPythonTools.overpass import Overpass
overpass = Overpass()

lib_data = overpass.query(library_query)

The variable `lib_data` now already contains the result from the query against OSM. Let's have a look at it. With `nodes()` we can access the retrieved points. Let's take a look at the first entry:

In [10]:
lib_data.nodes()[0].tags()

{'addr:state': 'TX',
 'amenity': 'library',
 'ele': '163',
 'gnis:county_name': 'Travis',
 'gnis:feature_id': '2360810',
 'gnis:import_uuid': '57871b70-0100-4405-bb30-88b2e001a944',
 'gnis:reviewed': 'no',
 'name': 'Texas State Law Library',
 'source': 'USGS Geonames'}

Similarly, we can also access the geometry, which in this case is just a point:

In [11]:
lib_data.nodes()[0].geometry()

{"coordinates": [-97.741983, 30.276236], "type": "Point"}

Next, we use the compact form of a list comprehension to extract the libraries' names and coordinates:


In [12]:
libraries = [ (lib.tag("name"), lib.geometry() ) for lib in lib_data.nodes()]

… which we turn into a GeoDataFrame. By naming the second column `geometry` we indicate towards GeoPandas to interpret the coordinates as geographic locations:

In [13]:
libraries = gpd.GeoDataFrame(libraries, columns = ['name', 'geometry'])
libraries.head()

,name,geometry
0,Texas State Law Library,POINT (-97.74198 30.27624)
1,Milwood Branch Austin City Library,POINT (-97.71617 30.42232)
2,Little Walnut Creek Branch Austin City Library,POINT (-97.69846 30.36327)
3,Southeast Austin Branch Austin City Library,POINT (-97.74193 30.18760)
4,Munday Library,POINT (-97.75750 30.23110)


## 🗺  Present

When we have geospatial data readily available as GeoDataFrames, we can now map them with Altair. 

(There are other mapping libraries for Python, such as [Folium](https://python-visualization.github.io/folium/), that provide additional functionalities. Altair's geovis features are basic, but do provide some variety of techniques and have the benefit to work consistently with the other chart types we covered.)


### Markers on maps

A simple start is placing locations on a base map and adding a bit of further information via tooltips. Let's do this with Austin's schools! First, we can have another look at the attributes:

In [14]:
schools.head()

,FID,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,...,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR,geometry
0,81891,480000811280,4800008,ROOSTER SPRINGS EL,48,1001 BELTERRA DR,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-97.98292 30.19218)
1,81892,480000813086,4800008,SYCAMORE SPRINGS EL,48,14451 SAWYER RANCH RD,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-98.00025 30.17858)
2,81893,480000813151,4800008,SYCAMORE SPRINGS MIDDLE,48,14451 SAWYER RANCH RD,AUSTIN,TX,78737,48,...,1,N,N,N,N,4825,48045,48025,2018-2019,POINT (-98.00025 30.17858)
3,81942,480001609410,4800016,AUSTIN CAN ACADEMY,48,2406 ROSEWOOD AVE,AUSTIN,TX,78702,48,...,1,N,N,N,N,4825,48046,48014,2018-2019,POINT (-97.70895 30.27222)
4,82016,480004408055,4800044,WAYSIDE EDEN PARK ACADEMY,48,6215 MANCHACA RD,AUSTIN,TX,78745,48,...,1,N,N,N,N,4821,48049,48014,2018-2019,POINT (-97.80292 30.20907)


We will now create a simple map with markers in the form of an  Altair chart consisting of two layers:

1.   The `neighborhoods` form the lower layer representing their boundaries and the overall geographic shape of Austin
2.   The `schools` are the points of interests that are displayed on top

When putting the two layers together they should actually refer to the same geographic location to make sense. Here the neighborhoods and schools both refer to Austin. Also note that the order when the charts are added together determines the vertical order: first the basemap and then markers on top.

In [15]:
# 1.  mark_geoshape transparently uses the geometry column
basemap = alt.Chart(neighborhoods).mark_geoshape(
    # add some styling to reduce the salience of the basemap
    fill="lightgray", stroke="darkgray",
).encode(
    tooltip = ['name'],
).properties(width=600, height=600)

# 2.  we use mark_circle to have more control over visual variables
markers = alt.Chart(schools).mark_circle(opacity=1).encode(
    # point latitude & longitude to coordinates in geometry column
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
    tooltip=['NAME', 'STREET', 'ZIP'],
)

# combine the two layers 
basemap + markers

alt.LayerChart(...)

### Dot density maps

Let's use the open maps data set again, and plot New York, NY's trees. Note, I had to disable the max rows since therea are more than 5,000 trees returned from the query.  Note that you can make out Manhattan even without a base layer.  

In [16]:
# 1. prepare query (and directly include the location lookup)
tree_query = overpassQueryBuilder(
    area=nominatim.query('New York, NY').areaId(),
    elementType='node',
    selector='"natural"="tree"', 
    out='body', 
    includeGeometry=True
)

# 2. execute query (and give it a bit more time to finish)
tree_data = overpass.query(tree_query, timeout=60)

# 3. get ids and coordinates of trees
tree_locs = [ (tree.id(), tree.geometry()) for tree in tree_data.nodes()]

# 4. create GeoDataFrame
trees = gpd.GeoDataFrame(tree_locs, columns=["id", "geometry"])

trees.head()

,id,geometry
0,207694783,POINT (-73.96385 40.66462)
1,726428014,POINT (-73.95597 40.80889)
2,793886267,POINT (-74.01516 40.70238)
3,1201708558,POINT (-73.93184 40.85510)
4,1201708559,POINT (-73.93239 40.85584)


In [17]:
alt.data_transformers.disable_max_rows()
treemap = alt.Chart(trees).mark_circle(
    # reduce the visual presence of each element
    size=5,
    # with a low dot opacity we can use overplotting to indicate densities
    opacity=.25,
    # a natural choice
    color="green"
).encode(
    longitude='geometry.coordinates[0]:Q', 
    latitude='geometry.coordinates[1]:Q'    
).properties(width=600, height=600)

treemap

alt.Chart(...)

### Choropleth maps

Finally, let's create the geovisualization that uses the fill color of geospatial shapes to encode quantitative data. To illustrate this, we will visualize the population densities around the world. We will use area and population information from GeoNames and get the geographic shapes of countries from a geojson file.

In [18]:
# load country data from geonames 
geonames = pd.read_csv("https://www.geonames.org/countryInfoCSV", sep='\t')
# select four columns
geonames = geonames[['name', 'iso alpha3', 'areaInSqKm', 'population']]
# set index to country code
geonames = geonames.set_index("iso alpha3")

geonames.head()

,name,areaInSqKm,population
iso alpha3,,,
AND,Andorra,468.0,77006
ARE,United Arab Emirates,82880.0,9630959
AFG,Afghanistan,647500.0,37172386
ATG,Antigua and Barbuda,443.0,96286
AIA,Anguilla,102.0,13254


Next we collect the geographic boundaries and `simplify` them a bit, as they have more detail than what we need here:

In [19]:
# load country's polygons from datahub
polygons = gpd.read_file("https://gist.githubusercontent.com/TieJean/f739f67075108868059b101a709a738f/raw/cb358884e028658993c2da1dfd19854e6c5b6c3b/countries.geo.json")
# remove country names, as we have them already
polygons = polygons.drop(columns=["name"])
# set index to country code
polygons = polygons.set_index("id")
# reduce the complexity of the shapes
polygons.geometry = polygons.geometry.simplify(.1)

polygons.head(20)

,geometry
id,
AFG,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
AGO,"MULTIPOLYGON (((16.32653 -5.87747, 16.86019 -7..."
ALB,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
ARE,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
ARG,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."
ARM,"POLYGON ((43.58275 41.09214, 44.97248 41.24813..."
ATA,"MULTIPOLYGON (((-59.57209 -80.04018, -60.15966..."
ATF,"POLYGON ((68.93500 -48.62500, 69.58000 -48.940..."
AUS,"MULTIPOLYGON (((145.39798 -40.79255, 146.36412..."


As both DataFrames use the three-letter country codes as indices we can join them like this (join uses the index by default, so we don't have to specify what to join on):

In [20]:
# inner means that we keep only those countries
# for which we have geometric and attribute data
countries = polygons.join(geonames, how='inner')

countries.tail()

,geometry,name,areaInSqKm,population
VUT,"MULTIPOLYGON (((167.84488 -16.46633, 167.51518...",Vanuatu,12200.0,292680
YEM,"POLYGON ((53.10857 16.65105, 52.38521 16.38241...",Yemen,527970.0,28498687
ZAF,"POLYGON ((31.52100 -29.25739, 30.05572 -31.140...",South Africa,1219912.0,57779622
ZMB,"POLYGON ((32.75937 -9.23060, 33.23139 -9.67672...",Zambia,752614.0,17351822
ZWE,"POLYGON ((31.19141 -22.25151, 29.43219 -22.091...",Zimbabwe,390580.0,14439018


Visualizing area or population in a choropleth map, arguably, makes little sense. So let's compute population densities:

In [21]:
countries["density"] = countries["population"] / countries["areaInSqKm"]
countries.head()

,geometry,name,areaInSqKm,population,density
AFG,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",Afghanistan,647500.0,37172386,57.409090
AGO,"MULTIPOLYGON (((16.32653 -5.87747, 16.86019 -7...",Angola,1246700.0,30809762,24.713052
ALB,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",Albania,28748.0,2866376,99.706971
ARE,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",United Arab Emirates,82880.0,9630959,116.203656
ARG,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",Argentina,2766890.0,44494502,16.081052


Keep only those countries with valid density value and turn these densities into integers:

In [22]:
countries = countries[countries['density'].notna()]
countries.density = countries.density.round(0).astype(int)

There is one ‘country’ that is not really one, which is Antarctica. We'll remove this from the list here.

In [23]:
countries = countries.drop("ATA")

Finally, we draw the chart using Altair's `mark_geoshape()` method. The distribution of densities is highly skewed, due to very small countries with relatively high population numbers, such as Monaco. To spread out the low and high density values we use a logarithmic scale and set the domain between 1 and 1000. Note that the domain has to end in a multiple of the base, which is by default 10.

In [24]:
alt.Chart(countries).mark_geoshape().encode(
    color=alt.Color('density', scale=alt.Scale(type="log", domain=[1,1000] )),
    tooltip=['name', 'areaInSqKm', 'population', 'density']
).properties(
    width=800,
    height=600
)

alt.Chart(...)

The map is shown in the default Mercator projection, which particularly distorts the area sizes of North America, Europe and Russia in contrast to Africa, Southern Asia and parts of South America.

💡 *You can change the projection used above to one that does not distort area sizes as much ([see this list for options](https://vega.github.io/vega-lite/docs/projection.html#projection-types)).* 

## Your Turn

Create the following:

   1. A visualization with Austin's neighborhoods overlayed with Austin's libraries. The tool tip should provide necessary information to identify each neighborhood and library.  Do you think the open data source is reliable? 
   2. Add the location of all the trees in Austin (according to Open Street Map) to the first visualization
   3. A population density choropleth with the Albers map projection

In [25]:
# 1.
basemap = alt.Chart(neighborhoods).mark_geoshape(
    fill="lightgray", stroke="darkgray",
).encode(
    tooltip = ['name'],
).properties(width=600, height=600)

markers = alt.Chart(libraries).mark_circle(opacity=1).encode(
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
    tooltip=['name'],
)

basemap + markers

alt.LayerChart(...)

In [26]:
tree_query = overpassQueryBuilder(
    area=nominatim.query('Austin, TX').areaId(),
    elementType='node',
    selector='"natural"="tree"', 
    out='body', 
    includeGeometry=True
)

# 2. execute query (and give it a bit more time to finish)
tree_data = overpass.query(tree_query, timeout=60)

# 3. get ids and coordinates of trees
tree_locs = [ (tree.id(), tree.geometry()) for tree in tree_data.nodes()]

# 4. create GeoDataFrame
trees = gpd.GeoDataFrame(tree_locs, columns=["id", "geometry"])

trees.head()

,id,geometry
0,1079727457,POINT (-97.73878 30.27459)
1,1079727564,POINT (-97.73996 30.27296)
2,1079727579,POINT (-97.74045 30.27282)
3,1079727594,POINT (-97.74033 30.27275)
4,1079727602,POINT (-97.73910 30.27508)


In [27]:
# 2.  
nbhoods = alt.Chart(neighborhoods).mark_geoshape(
    fill="lightgray", stroke="darkgray",
).encode(
    tooltip = ['name'],
).properties(width=600, height=600)

libpts = alt.Chart(libraries).mark_circle(opacity=1).encode(
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
    tooltip=['name'],
)

treepts = alt.Chart(trees).mark_circle(opacity=1, size=4, color='green').encode(
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
)

nbhoods + libpts + treepts

alt.LayerChart(...)

In [28]:
# 3
alt.Chart(countries).mark_geoshape().encode(
    color=alt.Color('density', scale=alt.Scale(type="log", domain=[1,1000] )),
    tooltip=['name', 'areaInSqKm', 'population', 'density']
).properties(
    width=800,
    height=600
).project(
    type='albers'
)

alt.Chart(...)

## Sources

Tutorials & Documentation
- [Specifying Geospatial Data in Altair — Altair 4.1.0 documentation](https://altair-viz.github.io/user_guide/data.html#geospatial-data)
- [GeoPandas](https://geopandas.org)
- [OSMPythonTools](https://github.com/mocnik-science/osm-python-tools)

Data
- [OpenStreetMap](https://www.openstreetmap.org/)
- [GeoNames](https://www.geonames.org)
- [Data.gov](https://catalog.data.gov/dataset/public-school-locations-current)
